In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torch.utils.data

In [ ]:
class VideoClipLoader(torch.utils.data.Dataset):
    def __init__(self, video_dir, metadata, pattern_vid = ".mp4", every = 25 ):
        
        
        self.every = every

        
        self.metadata = metadata
        #  make a list of all the videos available from the given metadata dataset
        self.video_list = video_dir + "/" + metadata["Folder name"].astype(str) + "/" + metadata["Clip Name"].astype(str)  + pattern_vid
                   

    # load the full video and separate into images
    def load_video_as_images(self, video_path):
        
        vc = cv2.VideoCapture(video_path)
                
        image_list = []
        
        index = 0
        while(vc.isOpened()):
            # Read a frame from video
            ret, frame = vc.read()

            if frame is None:
                break
                
            if index % self.every == 0: # check if the frame is the every other one
                frame = frame[:,:,0] # get only first dimension, they are the same
                image_list.append(frame)    
            
            index+=1
                
        
        vc.release()
                
        image_array = np.array(image_list)
        return image_array, video_path
    
    # get the metadata for the video clip
    def get_video_metadata(self, video_path):
        
        #  get the video path and slipt it
        video_path_split = video_path.split("/")
        
        #  extract the folder name and clip name 
        video_folder = video_path_split[-2]
        video_file = video_path_split[-1].split(".")[0]
        
        #  select the specified metadata row
        curr_metadata = self.metadata[(self.metadata.iloc[:,0]== int(video_folder))&(self.metadata.iloc[:,1]== video_file)].dropna()
        
        
        
        return curr_metadata
        

    #  get the new n images a video clip
    def __getitem__(self, idx):
        
        #  get images and path
        img, path = self.load_video_as_images(self.video_list.iloc[idx])
        
        
        #  get metadata
        metadata_curr = self.get_video_metadata(self.video_list.iloc[idx])
        
        # transform the images
        img = img / 255.0
        img = torch.from_numpy(img)
        img = img.float()
        img = torch.unsqueeze(img, 0)
        
        img = img.permute(1, 0, 2, 3)
        
        # transform the metadata to a string 
        output_metadata_list = metadata_curr.squeeze().values.tolist()
        output_metadata_list[2] = output_metadata_list[2].strftime("%Y-%m-%d %H:%M:%S")
        
        output_metadata_str = ','.join(map(str, output_metadata_list)) 
        
        return img, path ,output_metadata_str

    def __len__(self):
        return len(self.video_list)

In [ ]:



'''
METADATA structure:

    Folder name - name of the day folder in the format of YYYYMMDD
    Clip Name - name of the clip folderin the format clip_{number}_{HHMM}
    DateTime - the date time when the 2 min has started
    Temperature - in C
    Humidity - in %
    Precipitation - in kg/m2
    Dew Point - in C
    Wind Direction - in degrees
    Wind Speed - in m/s2
    Sun Radiation Intensity - W/m2
    Min of sunshine latest 10 min - in minutes

'''

'''
Config file structure:

    video_path - top path that contains all day video folders and the metadata csv
    metadata - the selected metadata dataframe that will be used to select the clips
    main_save_path - the top path for saving the created images and metadata
    every - sampling rate at which the images will be created from the video, default one is every 25th
    pattern_video - what pattern to be searched for in the folders, default one is .mp4

'''


# Script to get the queried metadata and create image set for it
def make_imagesets(cfg, save = False):

    # get the dataset from the selected metadata and the specified "every" next one
    dataset = VideoClipLoader(video_dir=cfg['video_path'], metadata = cfg['metadata'], every = cfg['every'])


    # create the folder
    if not os.path.exists(cfg['main_save_path']):

        os.makedirs(cfg['main_save_path'])

    #  go through all the clips in the dataset, cut them in images and save them
    metadata_list = []
    for i, sample in enumerate(dataset):
        img, path, curr_metadata = sample
        curr_metadata = curr_metadata.split(',')
        #  save dir for all the images from each clip
        save_dir = os.path.join(cfg['main_save_path'],curr_metadata[0],curr_metadata[1])

        #  create the save dir
        if save:
            if not os.path.exists(save_dir):

                os.makedirs(save_dir)

        #  go through all the images from the clip, transform them and save them to disk
        count = 0
        for sub_img in img:
            sub_img = sub_img.unsqueeze(0)
            input_img = sub_img.squeeze(1).mul(255).byte().numpy()

            count_str ="image_" + str(count).zfill(4)
            
            if save:
                cv2.imwrite(os.path.join(save_dir, count_str + '.jpg'), np.squeeze(input_img))
            #  create a metadata entry for the image, containing the image number
            img_metadata = [curr_metadata[0], curr_metadata[1], count_str, curr_metadata[2], curr_metadata[3], curr_metadata[4], curr_metadata[5], curr_metadata[6],curr_metadata[7], curr_metadata[8],curr_metadata[9],curr_metadata[10]]
            metadata_list.append(img_metadata)


            count+=1

        print(f"Finished {save_dir}")

    # save all the new metadata entries per image to a new csv in the same folder
    metadata_df = pd.DataFrame.from_records(metadata_list, columns=['Folder name', 'Clip Name', 'Image Number', 'DateTime', 'Temperature', 'Humidity', 'Precipitation', 'Dew Point', 'Wind Direction', 'Wind Speed', 'Sun Radiation Intensity', 'Min of sunshine latest 10 min'])
    
    if save:
        metadata_df.to_csv(os.path.join(cfg['main_save_path'], "metadata_images.csv"), index=False)



if __name__ == '__main__':

    every = 25

    pattern_vid = ".mp4"

    video_path = r"/kaggle/input/longterm-thermal-drift-dataset/LTD Dataset/Video Clips"

    metadata_name = r"/kaggle/input/longterm-thermal-drift-dataset/LTD Dataset/metadata.csv"
    metadata_path = os.path.join(video_path, metadata_name)

    metadata = pd.read_csv(metadata_path)
    metadata["DateTime"] = pd.to_datetime(metadata['DateTime'], dayfirst = True)


    main_save_dir = r"Image Dataset"
    
    

    # Training
    #  creating the cfg file containing all the necessary information for creating the training set
    cfg = {
        'video_path': video_path,
        'metadata': metadata,
        'main_save_path': main_save_dir,
        'every': every,
        'pattern_video': pattern_vid,

    }
    #  call the function
    make_imagesets(cfg, save = False)